In [1]:
import os
import json
import pandas as pd

from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter


In [2]:
# Paths
INPUT_JSON = "output/3_test_confessions_of_hnmu.json"
OUTPUT_XLSX = "output/3_test_confessions_of_hnmu.xlsx"

# Ensure output directory exists
os.makedirs(os.path.dirname(OUTPUT_XLSX) or ".", exist_ok=True)


In [3]:
# Excel safety helpers
# Excel treats values starting with "=", "+", "-", "@" as formulas.
# Social-media text can legitimately start with these characters (e.g., "=)))", "- hỏi chút"),
# causing Excel to "repair" the file and remove those formulas.
DANGEROUS_PREFIXES = ("=", "+", "-", "@")

def excel_safe(value):
    if isinstance(value, str) and value.startswith(DANGEROUS_PREFIXES):
        return "'" + value  # leading apostrophe forces plain-text in Excel
    return value


In [4]:
# Load JSON and flatten segments into rows
with open(INPUT_JSON, "r", encoding="utf-8") as f:
    data = json.load(f)

rows = []
for r in data:
    idx = r.get("index")
    page_url = r.get("page_url")
    post_url = r.get("post_url")
    post_text = r.get("post_text")

    segs = r.get("segments") or []
    if len(segs) == 0:
        # Keep at least one row so the post isn't lost
        segs = [post_text or ""]

    for j, seg in enumerate(segs, start=1):
        rows.append({
            "index": idx,
            "segment_no": j,
            "segment_text": seg,
            "post_url": post_url,
            "page_url": page_url,
            "post_text": post_text,
            "num_segments_in_post": len(r.get("segments") or []),
        })

df = pd.DataFrame(rows)

# Apply Excel-safe escaping to text columns (prevents "Removed Records: Formula" repairs)
TEXT_COLS = ["segment_text", "post_text", "post_url", "page_url"]
for col in TEXT_COLS:
    if col in df.columns:
        df[col] = df[col].astype(str).map(excel_safe)

df.head()


,index,segment_no,segment_text,post_url,page_url,post_text,num_segments_in_post
0,1,1,Hôm nay e có nhặt đc thẻ sinh viên này bên ghế...,https://www.facebook.com/permalink.php?story_f...,https://www.facebook.com/profile.php?id=615552...,Hôm nay e có nhặt đc thẻ sinh viên này bên ghế...,1
1,2,1,"12297: Tha thiết tìm roomate, cách cơ sở 3 100...",https://www.facebook.com/permalink.php?story_f...,https://www.facebook.com/profile.php?id=615552...,"12297: Tha thiết tìm roomate, cách cơ sở 3 100...",4
2,2,2,12298: Có ai tìm người ở ghép gần cơ sở 3 khôn...,https://www.facebook.com/permalink.php?story_f...,https://www.facebook.com/profile.php?id=615552...,"12297: Tha thiết tìm roomate, cách cơ sở 3 100...",4
3,2,3,12312: Tìm người ở ghép gần cs1 phòng 3tr5 chư...,https://www.facebook.com/permalink.php?story_f...,https://www.facebook.com/profile.php?id=615552...,"12297: Tha thiết tìm roomate, cách cơ sở 3 100...",4
4,2,4,#hnmutimtro #hnmucfs,https://www.facebook.com/permalink.php?story_f...,https://www.facebook.com/profile.php?id=615552...,"12297: Tha thiết tìm roomate, cách cơ sở 3 100...",4


In [5]:
# Export to XLSX
with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as writer:
    df.to_excel(writer, index=False, sheet_name="segments")


In [6]:
# Optional formatting for readability
wb = load_workbook(OUTPUT_XLSX)
ws = wb["segments"]

# Freeze header row
ws.freeze_panes = "A2"

# Header style
header_font = Font(bold=True)
for cell in ws[1]:
    cell.font = header_font
    cell.alignment = Alignment(vertical="center", wrap_text=True)

wrap_cols = {"segment_text", "post_text", "post_url", "page_url"}
col_names = [c.value for c in ws[1]]

for col_idx, name in enumerate(col_names, start=1):
    letter = get_column_letter(col_idx)

    # Column widths (heuristic)
    if name in ("index", "segment_no", "num_segments_in_post"):
        ws.column_dimensions[letter].width = 18
    elif name in ("post_url", "page_url"):
        ws.column_dimensions[letter].width = 45
    else:
        ws.column_dimensions[letter].width = 70

    # Wrap text where useful
    if name in wrap_cols:
        for cell in ws[letter][1:]:  # skip header
            cell.alignment = Alignment(wrap_text=True, vertical="top")

wb.save(OUTPUT_XLSX)
print("Saved:", OUTPUT_XLSX)


Saved: output/3_test_confessions_of_hnmu.xlsx
